In [28]:
import numpy as np
import torch
import pandas as pd
import os

import torch.nn as nn
from torch import optim
from torch.utils.data import TensorDataset, DataLoader
from missforest import MissForest
from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import warnings

In [63]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
mammographic_mass = fetch_ucirepo(id=161) 
  
# data (as pandas dataframes) 
X = mammographic_mass.data.features 
Y = mammographic_mass.data.targets 
  
# metadata 
print(mammographic_mass.metadata) 
  
# variable information 
print(mammographic_mass.variables) 

print(X.shape)


{'uci_id': 161, 'name': 'Mammographic Mass', 'repository_url': 'https://archive.ics.uci.edu/dataset/161/mammographic+mass', 'data_url': 'https://archive.ics.uci.edu/static/public/161/data.csv', 'abstract': "Discrimination of benign and malignant mammographic masses based on BI-RADS attributes and the patient's age.", 'area': 'Health and Medicine', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 961, 'num_features': 5, 'feature_types': ['Integer'], 'demographics': ['Age'], 'target_col': ['Severity'], 'index_col': None, 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 2007, 'last_updated': 'Thu Mar 28 2024', 'dataset_doi': '10.24432/C53K6Z', 'creators': ['Matthias Elter'], 'intro_paper': {'ID': 448, 'type': 'NATIVE', 'title': 'The prediction of breast cancer biopsy outcomes using two CAD approaches that both emphasize an intelligible decision process.', 'authors': 'M. Elter, R. Schulz-Wendtland, T. Wittenberg', 'v

In [64]:
Omega = X.isna().to_numpy()
Omega = Omega.astype(int)

X_ZI = X.fillna(0)
X_ZI = X_ZI.to_numpy()
scaler = StandardScaler()
scaler.fit(X_ZI)
X_ZI = scaler.transform(X_ZI)

sample_mean = X.mean()
X_MI = X.fillna(sample_mean)
scaler = StandardScaler()
scaler.fit(X_MI)
X_MI = scaler.transform(X_MI)

Y = Y.to_numpy().reshape(-1)

In [84]:
X_ZI_train, X_ZI_test, X_MI_train, X_MI_test, Y_train, Y_test, Omega_train, Omega_test = train_test_split(X_ZI, X_MI, Y, Omega, test_size=161, random_state=42)

In [85]:
X_ZI_train = torch.tensor(X_ZI_train, dtype=torch.float32)
X_ZI_test = torch.tensor(X_ZI_test, dtype=torch.float32)

X_MI_train = torch.tensor(X_MI_train, dtype=torch.float32)
X_MI_test = torch.tensor(X_MI_test, dtype=torch.float32)

Omega_train = torch.tensor(Omega_train, dtype=torch.float32)
Omega_test = torch.tensor(Omega_test, dtype=torch.float32)

Y_train = torch.tensor(Y_train, dtype=torch.long)

In [88]:
lr = 0.001
epochs = 200
l1_lambda = 0

class TwoStreamModel(nn.Module):
    def __init__(self):
        super().__init__()
        embedding_dim = 2
        
        # The embedding layer
        self.embedding = nn.Sequential(
            nn.Linear(5, 2),  
            nn.ReLU(),
            nn.Linear(2, embedding_dim),  
            nn.ReLU()
        )
        
        # Combined network for the layers after embedding
        self.combined = nn.Sequential(
            nn.Linear(5 + embedding_dim, 5),
            nn.ReLU(),
            nn.Linear(5, 5),
            nn.ReLU(),
            nn.Linear(5, 5),
            nn.ReLU(),
            nn.Linear(5, 5),
            nn.ReLU(),
            nn.Linear(5, 2)  # Output layer
        )
    
    def forward(self, x, omega):
        # Apply the embedding layer to omega
        embedded = self.embedding(omega)
        
        # Concatenate the embedding with x
        combined_features = torch.cat((x, embedded), dim=1)
        
        # Apply the combined network
        final_out = self.combined(combined_features)
        
        return final_out
    

model_PE = TwoStreamModel()
PE_train_data = TensorDataset(X_ZI_train, Omega_train, Y_train)
PE_train_loader = DataLoader(dataset = PE_train_data, batch_size=32, shuffle=True)

optimizer = optim.Adam(model_PE.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
loss_fn = nn.CrossEntropyLoss()   

for epoch in range(epochs):

    for x_batch, omega_batch, y_batch in PE_train_loader:
        optimizer.zero_grad()
        pred = model_PE(x_batch, omega_batch)
        loss = loss_fn(pred, y_batch)

        loss.backward()
        optimizer.step()

    scheduler.step()

    if epoch % 10 == 9:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

_, labels = torch.max(model_PE(X_ZI_test, Omega_test), dim=1)
pred = labels.detach().numpy()
np.mean(np.abs(pred - Y_test))

Epoch 9, Loss: 0.5106790065765381
Epoch 19, Loss: 0.3638306260108948
Epoch 29, Loss: 0.3738839626312256
Epoch 39, Loss: 0.2965232729911804
Epoch 49, Loss: 0.31402263045310974
Epoch 59, Loss: 0.3274551331996918
Epoch 69, Loss: 0.4839431941509247
Epoch 79, Loss: 0.397629976272583
Epoch 89, Loss: 0.48749831318855286
Epoch 99, Loss: 0.29553094506263733
Epoch 109, Loss: 0.6002846360206604
Epoch 119, Loss: 0.3556693494319916
Epoch 129, Loss: 0.355730801820755
Epoch 139, Loss: 0.3913477063179016
Epoch 149, Loss: 0.4459676742553711
Epoch 159, Loss: 0.3971387445926666
Epoch 169, Loss: 0.4150277078151703
Epoch 179, Loss: 0.37465810775756836
Epoch 189, Loss: 0.29482516646385193
Epoch 199, Loss: 0.6196262240409851


np.float64(0.16770186335403728)

In [89]:
lr = 0.001
epochs = 200
l1_lambda = 0

class FCModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(5, 5),
            nn.ReLU(),
            nn.Linear(5, 5),
            nn.ReLU(),
            nn.Linear(5, 5),
            nn.ReLU(),
            nn.Linear(5, 5),
            nn.ReLU(),
            nn.Linear(5, 2)  # Output layer
        )
    
    def forward(self, x):   
        final_out = self.fc(x)
        return final_out
    

model_MI = FCModel()
MI_train_data = TensorDataset(X_MI_train, Y_train)
MI_train_loader = DataLoader(dataset = MI_train_data, batch_size=32, shuffle=True)

optimizer = optim.Adam(model_MI.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
loss_fn = nn.CrossEntropyLoss()   

for epoch in range(epochs):

    for x_batch, y_batch in MI_train_loader:
        optimizer.zero_grad()
        pred = model_MI(x_batch)
        loss = loss_fn(pred, y_batch)

        loss.backward()
        optimizer.step()

    scheduler.step()

    if epoch % 10 == 9:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

_, labels = torch.max(model_MI(X_MI_test), dim=1)
pred = labels.detach().numpy()
np.mean(np.abs(pred - Y_test))

Epoch 9, Loss: 0.6302314400672913
Epoch 19, Loss: 0.49396929144859314
Epoch 29, Loss: 0.5680058598518372
Epoch 39, Loss: 0.4505730867385864
Epoch 49, Loss: 0.457862913608551
Epoch 59, Loss: 0.4324200451374054
Epoch 69, Loss: 0.45647433400154114
Epoch 79, Loss: 0.5443997383117676
Epoch 89, Loss: 0.5014747977256775
Epoch 99, Loss: 0.37881484627723694
Epoch 109, Loss: 0.49651721119880676
Epoch 119, Loss: 0.4855366349220276
Epoch 129, Loss: 0.4757787883281708
Epoch 139, Loss: 0.5127899050712585
Epoch 149, Loss: 0.4404394030570984
Epoch 159, Loss: 0.6043076515197754
Epoch 169, Loss: 0.37158384919166565
Epoch 179, Loss: 0.377371609210968
Epoch 189, Loss: 0.38464754819869995
Epoch 199, Loss: 0.5572265386581421


np.float64(0.17391304347826086)